In [1]:
from ipynb.fs.full.utility import *
from ipynb.fs.full.dataset import *

In [2]:
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt 
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
# hyper params
input_size = 138
hidden_size = 200
output_size = 25
num_epochs = 100
BATCH_SIZE = 512
learning_rate = 1e-3

In [4]:
# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")     
else:
    device = torch.device("cpu")

In [5]:
torch.manual_seed(0)

In [6]:
beatles_dataset = BeatlesDataset(npzs_8th)
train_loader, val_loader, test_loader = datasetSplit(beatles_dataset, batch_size=BATCH_SIZE)

In [7]:
len(train_loader)

741

In [8]:
len(val_loader)

92

In [9]:
len(test_loader)

92

In [10]:
for x, y in train_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([512, 123])
torch.Size([512, 25])


In [8]:
'''
Reference:
https://www.youtube.com/watch?v=0_PgWWmauHk
https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
'''

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        hidden = self.init_hidden()

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)
        out = self.softmax(out)
        
        return out, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size).to(device)

rnn = RNN(input_size, hidden_size, output_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    n_total_steps = len(train_loader)
    for i, (x, y) in enumerate(train_loader):
        
        out, _ = rnn(torch.Tensor(x))
        loss = loss_fn(out, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
'''
plt.figure()
plt.plot(all_losses)
plt.show()

def predict(input_line):
    print(f"\n> {input_line}")
    with torch.no_grad():
        line_tensor = line_to_tensor(input_line)
        
        hidden = rnn.init_hidden()
    
        for i in range(line_tensor.size()[0]):
            output, hidden = rnn(line_tensor[i], hidden)
        
        guess = category_from_output(output)
        print(guess)


while True:
    sentence = input("Input:")
    if sentence == "quit":
        break
    
    predict(sentence)
    '''

Epoch [1/100], Step [100/741], Loss: 2.8548
Epoch [1/100], Step [200/741], Loss: 2.7656
Epoch [1/100], Step [300/741], Loss: 2.7196
Epoch [1/100], Step [400/741], Loss: 2.7537
Epoch [1/100], Step [500/741], Loss: 2.6810
Epoch [1/100], Step [600/741], Loss: 2.6995
Epoch [1/100], Step [700/741], Loss: 2.7437
Epoch [2/100], Step [100/741], Loss: 2.7087
Epoch [2/100], Step [200/741], Loss: 2.7178
Epoch [2/100], Step [300/741], Loss: 2.6999
Epoch [2/100], Step [400/741], Loss: 2.6683
Epoch [2/100], Step [500/741], Loss: 2.6342
Epoch [2/100], Step [600/741], Loss: 2.5534
Epoch [2/100], Step [700/741], Loss: 2.6284
Epoch [3/100], Step [100/741], Loss: 2.6842
Epoch [3/100], Step [200/741], Loss: 2.4892
Epoch [3/100], Step [300/741], Loss: 2.5360
Epoch [3/100], Step [400/741], Loss: 2.5565
Epoch [3/100], Step [500/741], Loss: 2.5429
Epoch [3/100], Step [600/741], Loss: 2.6225
Epoch [3/100], Step [700/741], Loss: 2.5806
Epoch [4/100], Step [100/741], Loss: 2.6125
Epoch [4/100], Step [200/741], L

Epoch [27/100], Step [300/741], Loss: 2.2418
Epoch [27/100], Step [400/741], Loss: 2.3604
Epoch [27/100], Step [500/741], Loss: 2.3303
Epoch [27/100], Step [600/741], Loss: 2.3137
Epoch [27/100], Step [700/741], Loss: 2.2952
Epoch [28/100], Step [100/741], Loss: 2.1758
Epoch [28/100], Step [200/741], Loss: 2.2786
Epoch [28/100], Step [300/741], Loss: 2.3151
Epoch [28/100], Step [400/741], Loss: 2.2644
Epoch [28/100], Step [500/741], Loss: 2.3644
Epoch [28/100], Step [600/741], Loss: 2.3268
Epoch [28/100], Step [700/741], Loss: 2.3892
Epoch [29/100], Step [100/741], Loss: 2.3331
Epoch [29/100], Step [200/741], Loss: 2.3383
Epoch [29/100], Step [300/741], Loss: 2.3397
Epoch [29/100], Step [400/741], Loss: 2.2835
Epoch [29/100], Step [500/741], Loss: 2.2813
Epoch [29/100], Step [600/741], Loss: 2.3576
Epoch [29/100], Step [700/741], Loss: 2.3515
Epoch [30/100], Step [100/741], Loss: 2.2398
Epoch [30/100], Step [200/741], Loss: 2.2427
Epoch [30/100], Step [300/741], Loss: 2.3141
Epoch [30/

KeyboardInterrupt: 

In [8]:
'''Ref: 
https://www.youtube.com/watch?v=0_PgWWmauHk
https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/'''

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        hidden = self.init_hidden()

        # Passing in the input and hidden state into the model and obtaining outputs
        
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc(out)
        out = self.softmax(out)
        
        return out, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size).to(device)

rnn = RNN(input_size, hidden_size, output_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    n_total_steps = len(train_loader)
    for i, (x, y) in enumerate(train_loader):
        
        out, _ = rnn(torch.Tensor(x))
        loss = loss_fn(out, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
'''
plt.figure()
plt.plot(all_losses)
plt.show()

def predict(input_line):
    print(f"\n> {input_line}")
    with torch.no_grad():
        line_tensor = line_to_tensor(input_line)
        
        hidden = rnn.init_hidden()
    
        for i in range(line_tensor.size()[0]):
            output, hidden = rnn(line_tensor[i], hidden)
        
        guess = category_from_output(output)
        print(guess)


while True:
    sentence = input("Input:")
    if sentence == "quit":
        break
    
    predict(sentence)
    '''

Epoch [1/100], Step [100/741], Loss: 2.6778
Epoch [1/100], Step [200/741], Loss: 2.7518
Epoch [1/100], Step [300/741], Loss: 2.8422
Epoch [1/100], Step [400/741], Loss: 2.7264
Epoch [1/100], Step [500/741], Loss: 2.7881
Epoch [1/100], Step [600/741], Loss: 2.7572
Epoch [1/100], Step [700/741], Loss: 2.7296
Epoch [2/100], Step [100/741], Loss: 2.6835
Epoch [2/100], Step [200/741], Loss: 2.7235
Epoch [2/100], Step [300/741], Loss: 2.7495
Epoch [2/100], Step [400/741], Loss: 2.5769
Epoch [2/100], Step [500/741], Loss: 2.6274
Epoch [2/100], Step [600/741], Loss: 2.6194
Epoch [2/100], Step [700/741], Loss: 2.6570
Epoch [3/100], Step [100/741], Loss: 2.6212
Epoch [3/100], Step [200/741], Loss: 2.6292
Epoch [3/100], Step [300/741], Loss: 2.5656
Epoch [3/100], Step [400/741], Loss: 2.5559
Epoch [3/100], Step [500/741], Loss: 2.5776
Epoch [3/100], Step [600/741], Loss: 2.5468
Epoch [3/100], Step [700/741], Loss: 2.5978
Epoch [4/100], Step [100/741], Loss: 2.6145
Epoch [4/100], Step [200/741], L

Epoch [27/100], Step [300/741], Loss: 2.3378
Epoch [27/100], Step [400/741], Loss: 2.3550
Epoch [27/100], Step [500/741], Loss: 2.2282
Epoch [27/100], Step [600/741], Loss: 2.3621
Epoch [27/100], Step [700/741], Loss: 2.3460
Epoch [28/100], Step [100/741], Loss: 2.3159
Epoch [28/100], Step [200/741], Loss: 2.3212
Epoch [28/100], Step [300/741], Loss: 2.3930
Epoch [28/100], Step [400/741], Loss: 2.2344
Epoch [28/100], Step [500/741], Loss: 2.3585
Epoch [28/100], Step [600/741], Loss: 2.1600
Epoch [28/100], Step [700/741], Loss: 2.3013
Epoch [29/100], Step [100/741], Loss: 2.3187
Epoch [29/100], Step [200/741], Loss: 2.4267
Epoch [29/100], Step [300/741], Loss: 2.2999
Epoch [29/100], Step [400/741], Loss: 2.4115
Epoch [29/100], Step [500/741], Loss: 2.3640
Epoch [29/100], Step [600/741], Loss: 2.2440
Epoch [29/100], Step [700/741], Loss: 2.3684
Epoch [30/100], Step [100/741], Loss: 2.2136
Epoch [30/100], Step [200/741], Loss: 2.2794
Epoch [30/100], Step [300/741], Loss: 2.3339
Epoch [30/

Epoch [53/100], Step [400/741], Loss: 2.2937
Epoch [53/100], Step [500/741], Loss: 2.4034
Epoch [53/100], Step [600/741], Loss: 2.4441
Epoch [53/100], Step [700/741], Loss: 2.2569
Epoch [54/100], Step [100/741], Loss: 2.2175
Epoch [54/100], Step [200/741], Loss: 2.2150
Epoch [54/100], Step [300/741], Loss: 2.3139
Epoch [54/100], Step [400/741], Loss: 2.3023
Epoch [54/100], Step [500/741], Loss: 2.2957
Epoch [54/100], Step [600/741], Loss: 2.2646
Epoch [54/100], Step [700/741], Loss: 2.2902
Epoch [55/100], Step [100/741], Loss: 2.3517
Epoch [55/100], Step [200/741], Loss: 2.2951
Epoch [55/100], Step [300/741], Loss: 2.2828
Epoch [55/100], Step [400/741], Loss: 2.3077
Epoch [55/100], Step [500/741], Loss: 2.2905
Epoch [55/100], Step [600/741], Loss: 2.3460
Epoch [55/100], Step [700/741], Loss: 2.3280
Epoch [56/100], Step [100/741], Loss: 2.3282
Epoch [56/100], Step [200/741], Loss: 2.3187
Epoch [56/100], Step [300/741], Loss: 2.2889
Epoch [56/100], Step [400/741], Loss: 2.3656
Epoch [56/

KeyboardInterrupt: 

In [8]:
# fnn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(138, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 25),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
model = NeuralNetwork().to(device)

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
def train_loop(model, loss_fn, optimizer):
    size = len(train_loader)
    for e in range(num_epochs):
        for i, (X, y) in enumerate(train_loader):

            # Compute prediction and loss
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 100 == 0:
                loss, current = loss.item(), (i + 1)
                print(f"epoch {e}: loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
train_loop(model, loss_fn, optimizer)

epoch 0: loss: 3.230472  [    1/  741]
epoch 0: loss: 2.732932  [  101/  741]
epoch 0: loss: 2.713971  [  201/  741]
epoch 0: loss: 2.658298  [  301/  741]
epoch 0: loss: 2.683450  [  401/  741]
epoch 0: loss: 2.646077  [  501/  741]
epoch 0: loss: 2.651324  [  601/  741]
epoch 0: loss: 2.700296  [  701/  741]
epoch 1: loss: 2.579834  [    1/  741]
epoch 1: loss: 2.546856  [  101/  741]
epoch 1: loss: 2.573468  [  201/  741]
epoch 1: loss: 2.590904  [  301/  741]
epoch 1: loss: 2.547320  [  401/  741]
epoch 1: loss: 2.541674  [  501/  741]
epoch 1: loss: 2.533453  [  601/  741]
epoch 1: loss: 2.509900  [  701/  741]
epoch 2: loss: 2.495333  [    1/  741]
epoch 2: loss: 2.447051  [  101/  741]
epoch 2: loss: 2.528401  [  201/  741]
epoch 2: loss: 2.520875  [  301/  741]
epoch 2: loss: 2.409639  [  401/  741]
epoch 2: loss: 2.514967  [  501/  741]
epoch 2: loss: 2.524416  [  601/  741]
epoch 2: loss: 2.438348  [  701/  741]
epoch 3: loss: 2.381194  [    1/  741]
epoch 3: loss: 2.379784  

KeyboardInterrupt: 

In [20]:
X, y = test_dataset[0:512]

In [24]:
with torch.no_grad():
    out = model(X)
    print(out[0])

tensor([ 1.4344, -0.9752, -0.7345, -0.8160,  1.1610, -0.5414, -1.1247, -2.4593,
         0.9745, -0.2917, -0.3639, -1.0234, -0.1265, -0.6864,  1.0501, -1.9562,
        -0.1258, -2.5222,  1.3533, -0.0928, -0.1041, -1.4251,  0.3371, -0.3832,
        -0.2196], device='cuda:0')


In [36]:
commonIdxToChord(4)

'D:maj'

In [43]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for i, (X, y) in enumerate(test_loader):
        out = model(X)
        predicted = torch.argmax(out.data, 1)
        n_samples += y.size(0)
        label = torch.argmax(y, 1)
        n_correct += (predicted == label).sum().item()
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test image: {acc:.2f}%')

Accuracy of the network on the 10000 test image: 20.85%
